In [1]:
from tensorflow.keras.models import load_model

model = load_model("asl_vgg16_model.h5")
print("Model loaded successfully!")


Model loaded successfully!


In [2]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

# Load your saved VGG16 model
model = load_model("asl_resnet50v2_optimized.h5")

# ASL classes
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
           'W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # mirror view
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw landmarks on frame
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get bounding box of the hand
            h, w, _ = frame.shape
            x_min = w
            y_min = h
            x_max = y_max = 0

            for lm in hand_landmarks.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x)
                y_max = max(y_max, y)

            # Add some padding
            pad = 20
            x_min = max(x_min - pad, 0)
            y_min = max(y_min - pad, 0)
            x_max = min(x_max + pad, w)
            y_max = min(y_max + pad, h)

            # Crop hand region
            hand_img = frame[y_min:y_max, x_min:x_max]
            if hand_img.size != 0:
                hand_img = cv2.resize(hand_img, (32, 32))
                hand_img = hand_img.astype('float32') / 255.0
                hand_img = np.expand_dims(hand_img, axis=0)

                # Predict class
                prediction = model.predict(hand_img)
                class_idx = np.argmax(prediction, axis=1)[0]
                class_label = classes[class_idx]

                # Display prediction
                cv2.putText(frame, f'Prediction: {class_label}', (x_min, y_min - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("ASL Real-Time Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1764568713.106645 1850186 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1764568713.137465 1958014 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764568713.145591 1958014 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764568718.088760 1958019 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "vgg16" is incompatible with the layer: expected shape=(None, 128, 128, 3), found shape=(1, 32, 32, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 32, 32, 3), dtype=float32)
  • training=False
  • mask=None

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("asl_resnet50v2_optimized.h5")

# Your class labels
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']

input_shape = (128, 128)  # match the size used in training

# Open webcam
cap = cv2.VideoCapture(0)  # 0 = default camera

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Flip for mirror effect
    frame = cv2.flip(frame, 1)
    
    # Define ROI (Region of Interest) where hand should be
    x1, y1, x2, y2 = 100, 100, 400, 400
    roi = frame[y1:y2, x1:x2]
    
    # Preprocess ROI for model
    img = cv2.resize(roi, input_shape)
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)  # model expects batch dimension
    
    # Predict
    pred = model.predict(img)
    class_idx = np.argmax(pred)
    label = classes[class_idx]
    
    # Display
    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
    cv2.putText(frame, f"Prediction: {label}", (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (225, 0, 0), 2)
    
    cv2.imshow("ASL Hand Gesture Recognition", frame)
    
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━

KeyboardInterrupt: 

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet_v2 import preprocess_input 

# ======================
# 1. SETUP & VARIABLES
# ======================
print("Loading model... please wait.")
# Make sure the file name matches exactly what you saved earlier
model = load_model("asl_resnet50v2_optimized.h5") 

classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'nothing', 'space', 'del']

input_shape = (128, 128)

# Variables to hold text
current_word = ""
current_prediction = "..."

# ======================
# 2. MOUSE CALLBACK FUNCTION
# ======================
def mouse_click(event, x, y, flags, param):
    global current_word, current_prediction
    
    if event == cv2.EVENT_LBUTTONDOWN:
        # Check if "ADD LETTER" button is clicked
        if 660 < x < 840 and 250 < y < 300:
            if current_prediction not in ['nothing', 'del', 'space']:
                current_word += current_prediction
            elif current_prediction == 'space':
                current_word += " "
        
        # Check if "ADD SPACE" button is clicked
        elif 660 < x < 840 and 310 < y < 360:
            current_word += " "

        # Check if "BACKSPACE" button is clicked
        elif 660 < x < 840 and 370 < y < 420:
            current_word = current_word[:-1]

        # Check if "CLEAR ALL" button is clicked
        elif 660 < x < 840 and 430 < y < 480:
            current_word = ""

# ======================
# 3. MAIN LOOP
# ======================
# Open Webcam
cap = cv2.VideoCapture(0)

# Setup Window
cv2.namedWindow("ASL Translator")
cv2.setMouseCallback("ASL Translator", mouse_click)

print("Starting video stream... Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # FIX: Resize the frame to 640x480 so it fits our canvas perfectly
    frame = cv2.resize(frame, (640, 480))
    
    # Flip for mirror effect (natural interaction)
    frame = cv2.flip(frame, 1)
    
    # --- 1. DEFINE LAYOUT ---
    # Create a larger canvas: 480 height, 850 width (640 for cam + 210 for UI)
    canvas = np.zeros((480, 850, 3), dtype="uint8")
    canvas[:, :640] = frame # Paste webcam feed on the left
    
    # Draw UI Background (Dark Gray Panel on right)
    canvas[:, 640:] = (50, 50, 50) 
    
    # --- 2. PREDICTION LOGIC ---
    # Define ROI (Region of Interest) where hand should be
    x1, y1, x2, y2 = 100, 100, 400, 400
    roi = frame[y1:y2, x1:x2]
    
    # Draw ROI Box on feed (blue box)
    cv2.rectangle(canvas, (x1, y1), (x2, y2), (255, 0, 0), 2)
    
    if roi.size != 0:
        # Resize ROI to 128x128 for the ResNet model
        img = cv2.resize(roi, input_shape)
        img = img.astype("float32")
        
        # CRITICAL: Use ResNet specific preprocessing (-1 to 1 scaling)
        img = preprocess_input(img) 
        
        img = np.expand_dims(img, axis=0)
        
        # Predict
        pred = model.predict(img, verbose=0)
        class_idx = np.argmax(pred)
        current_prediction = classes[class_idx]
    
    # --- 3. DRAW UI ELEMENTS ---
    
    # [Prediction Display]
    cv2.putText(canvas, "Detected:", (650, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    cv2.putText(canvas, current_prediction, (650, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 3)

    # [The Sentence Box]
    cv2.putText(canvas, "Word/Sentence:", (650, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    cv2.rectangle(canvas, (650, 160), (840, 220), (255, 255, 255), -1) 
    display_text = current_word[-10:] # Show last 10 chars so it doesn't overflow
    cv2.putText(canvas, display_text, (655, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)

    # [Button 1: Add Letter (Green)]
    cv2.rectangle(canvas, (660, 250), (840, 300), (0, 200, 0), -1)
    cv2.putText(canvas, "ADD LETTER", (675, 285), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # [Button 2: Add Space (Blue)]
    cv2.rectangle(canvas, (660, 310), (840, 360), (200, 100, 0), -1)
    cv2.putText(canvas, "ADD SPACE", (680, 345), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # [Button 3: Backspace (Orange)]
    cv2.rectangle(canvas, (660, 370), (840, 420), (0, 100, 255), -1)
    cv2.putText(canvas, "BACKSPACE", (680, 405), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # [Button 4: Clear (Red)]
    cv2.rectangle(canvas, (660, 430), (840, 480), (0, 0, 255), -1)
    cv2.putText(canvas, "CLEAR ALL", (690, 465), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # --- 4. DISPLAY ---
    cv2.imshow("ASL Translator", canvas)
    
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Loading model... please wait.


Starting video stream... Press 'q' to quit.


KeyboardInterrupt: 